### Test the raster image dataloader

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader 
import numpy as np
import raster_relight as rr
import raster_dataloader as rd
import open3d as o3d
from matplotlib import pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ds = rd.RasterDataset()

loading combined from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_combined.png
loading normal from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_normal.png
loading albedo from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_albedo.png
loading depth from /home/dtetruash/Thesis/datasets/nerf-blender/nerf_synthetic/intrinsic_tester_sphere/r_000_depth.png
Got depth_alpha from 0 to 161 with mean of 63.2880296875. The datatype is uint8
Bit depth used for depth remapping was 8 and a max values of 255
Ater remapping got depth_remapped from 94 to 255 with mean of 191.7119703125. The datatype is uint8
After normalizing got depth_normalized from 0.3686274509803922 to 1.0 with mean of 0.7518116482843138. The datatype is float64
Occumpancy mask of shape (800, 800), and with 263345 occupied pixels.
Normal pixels range from 0 to 65535
Normals pixels shape: (263345, 3)
wo

In [3]:
dataset_list = list(ds)
len(dataset_list)

263345

In [4]:
normal = [sample[0] for sample in dataset_list]
albedo = [sample[1] for sample in dataset_list]
raster = [sample[2] for sample in dataset_list]


In [5]:
%%script echo skipping
albedo_img = np.ones((800,800,3)) * 255
albedo_img[ds._occupancy_mask] = np.array(albedo)
plt.imshow(albedo_img.astype(np.uint8))

skipping


In [6]:
%%script echo skipping
raster_img = np.ones((800,800,3)) * 255
raster_img[ds._occupancy_mask] = np.array(raster)
plt.imshow(raster_img.astype(np.uint8))

skipping


In [7]:
%%script echo skipping
normal_img = np.ones((800,800,3)) 
normal_img[ds._occupancy_mask] = (np.array(normal) * 0.5 + 0.5) 
plt.imshow(normal_img)

skipping


### Make sure the normals and light vectors are well aligned

In [8]:
pcd = ds._pcd

In [9]:
downpcd = pcd.voxel_down_sample(voxel_size=0.2)

In [10]:
o3d.visualization.draw_geometries([downpcd])

In [11]:
# make line set to the light point.

In [12]:
downpcd_points = np.asarray(downpcd.points)
len(downpcd_points)

205

In [13]:
pcd_points = np.asarray(ds._pcd.points)
pcd_points.shape

(263345, 3)

In [14]:
step = 35

In [15]:
ls_points_light = np.array([(i ,i + l*n) for (i,l,n) in zip(pcd_points, ds._light_vecs, ds._light_vecs_norms)][::step]).reshape(-1,3)
ls_points_light.shape

(15050, 3)

In [16]:
ls_edges_light = [(i, i+1) for i in range(0,len(ls_points_light),2) ][::step]
len(ls_edges_light)

215

In [17]:

line_set_light = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(ls_points_light),
    lines=o3d.utility.Vector2iVector(ls_edges_light),
)
line_set_light.colors = o3d.utility.Vector3dVector([(1,0,0)]*len(ls_edges_light))

In [18]:
ls_points_cam = np.array([(i ,i + l*n) for (i,l,n) in zip(pcd_points, ds._cam_vecs, ds._cam_vec_norms)][::step]).reshape(-1,3)
ls_points_cam.shape


(15050, 3)

In [19]:
ls_edges_cam = [(i, i+1) for i in range(0,len(ls_points_cam),2) ][::step]
len(ls_edges_cam)

215

In [20]:

line_set_cam = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(ls_points_cam),
    lines=o3d.utility.Vector2iVector(ls_edges_cam),
)
line_set_cam.colors = o3d.utility.Vector3dVector([(0,1,0)]*len(ls_edges_cam))

In [21]:
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=10)
mesh_sphere.compute_vertex_normals()
mesh_sphere.paint_uniform_color([0.1, 0.1, 0.7])

TriangleMesh with 182 points and 360 triangles.

In [22]:
sphere_line_set = o3d.geometry.LineSet.create_from_triangle_mesh(mesh_sphere)


In [23]:
sphere_line_set.colors = o3d.utility.Vector3dVector(np.array([[0,0,0]]*len(sphere_line_set.lines)))

In [24]:
# light point 
light_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
light_sphere.compute_vertex_normals()
light_sphere.paint_uniform_color([0.5,0.5,0])
light_sphere.translate([0,0,2])

TriangleMesh with 762 points and 1520 triangles.

In [31]:
cam_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
cam_sphere.compute_vertex_normals()
cam_sphere.paint_uniform_color([0.1,0.8,0.1])
cam_sphere.translate(ds._camera_center)

TriangleMesh with 762 points and 1520 triangles.

In [25]:
axis_set =  o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[v*(a:=10),-v*a] for v in np.eye(3)]).reshape(-1,3)),
    lines=o3d.utility.Vector2iVector([(i, i+1) for i in range(0,6,2) ]),
)
axis_set.colors = o3d.utility.Vector3dVector(np.eye(3))

In [ ]:
o3d.visualization.draw_geometries([downpcd,sphere_line_set, line_set_light,line_set_cam, light_sphere,cam_sphere, axis_set])

In [27]:
list(range(8))[::4]

[0, 4]

In [28]:
# make sphere point clou[d
basis = np.array([[0]*3]*3)
for i,b in enumerate(basis):
    b[i] +=1
basis

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])